In [1]:
# import io
import docx
import os
# import pytesseract
from tabulate import tabulate
from PIL import Image

In [2]:
import sys

In [3]:
sys.path.insert(0, "./../")

In [4]:
# doc_path = "./../documents/尼得科CT变频器推广使用说明.docx"
doc_path = "./../documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx"

In [5]:
doc = docx.Document(doc_path)

In [6]:
def extract_tables(doc):
    tables = []
    for table in doc.tables:
        table_data = []
        for row in table.rows:
            row_data = []
            for cell in row.cells:
                cell_text = ''
                for paragraph in cell.paragraphs:
                    cell_text += paragraph.text
                row_data.append(cell_text)
            table_data.append(row_data)

        tables.append(tabulate(table_data, tablefmt="plain"))
    return tables

In [7]:
tables = extract_tables(doc)

In [8]:
def extract_texts(doc):
    texts = []
    for p in doc.paragraphs:
        texts.append(p.text)
    return texts

In [9]:
elements = []
table_indices = []
for index, element in enumerate(doc.element.body):
    if element.tag.endswith("tbl"):
        table_indices.append(index)
    elements.append(element.text)

In [10]:
for index, table in enumerate(tables):
    table_index = table_indices[index]
    elements[table_index] = table

In [11]:
elements = [element for element in elements if isinstance(element, str)]

In [12]:
def consolidate_elements(elements, sentence_len=120, overlapping_size=2):

    final_sentence_list = []
    accumulate_len = 0
    windows_sentences = []

    for sentence in elements:

        word_len = len(sentence)
        if accumulate_len+word_len <= sentence_len:
            windows_sentences.append(sentence)
            accumulate_len += word_len
        else:
            windows_sentence = "\n".join(windows_sentences)
            final_sentence_list.append(windows_sentence)
            windows_sentences = windows_sentences[-overlapping_size:].copy()+[
                sentence]
            accumulate_len = word_len

    if len(windows_sentences) > 0:
        windows_sentence = "\n".join(windows_sentences)
        final_sentence_list.append(windows_sentence)

    return final_sentence_list

In [13]:
chunks = consolidate_elements(elements, sentence_len=256, overlapping_size=2)

In [14]:
# print(chunks[2])

In [15]:
from app.routers.documents.embedding import SentencesToVectors

In [16]:
sentences_to_vectors = SentencesToVectors()

/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [17]:
# sentences_to_vectors(chunks)

In [18]:
import chromadb

In [19]:
db_path = "/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/app/data"

In [20]:
chroma_client = chromadb.PersistentClient(path=db_path)

In [21]:
document_collection = chroma_client.get_or_create_collection(name="documents")

In [22]:
file_path = os.path.abspath(doc_path)
file_name = os.path.basename(doc_path)

In [28]:
document_collection.add(
    documents=[file_name],
    embeddings=[[0]],
    metadatas={"file_name": file_name, "file_path": file_path},
    ids=[file_name]
)

Add of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx
Insert of existing embedding ID: 健鼎机组蒸汽回水pH控制器设计更改事宜.docx


In [29]:
document_collection.get()

{'ids': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx', '尼得科CT变频器推广使用说明.docx'],
 'embeddings': None,
 'metadatas': [{'file_name': '健鼎机组蒸汽回水pH控制器设计更改事宜.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/健鼎机组蒸汽回水pH控制器设计更改事宜.docx'},
  {'file_name': '尼得科CT变频器推广使用说明.docx',
   'file_path': '/Users/nelsonlin/Desktop/workspaces/llm-local-chat-api/documents/尼得科CT变频器推广使用说明.docx'}],
 'documents': ['健鼎机组蒸汽回水pH控制器设计更改事宜.docx', '尼得科CT变频器推广使用说明.docx'],
 'uris': None,
 'data': None}

In [30]:
embedding_collection = chroma_client.get_or_create_collection(
    name="documents_embedding")

In [31]:
metadatas = [{'file_path': file_path, 'file_name': file_name}] * len(chunks)

In [32]:
embeddings = sentences_to_vectors(chunks)

In [33]:
ids = [file_name+f"_{i}" for i in range(len(chunks))]

In [41]:
len(embeddings)

4

In [43]:
embedding_collection.add(
    documents=chunks,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

In [ ]:
embedding_collection.query()